In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
import numpy as np

In [4]:
config=tf.ConfigProto()
config.gpu_options.allow_growth = True

In [5]:
tf.set_random_seed(777)

In [6]:
# parameters
log_dir = './logs/ML_Lab_11-07-Deep CNN'
learning_rate = 0.001
batch_size = 100
num_epochs = 50

In [7]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='X')

In [9]:
X_img = tf.reshape(X, shape=[-1, 28, 28, 1])

In [10]:
Y = tf.placeholder(shape=[None,  10], dtype=tf.float32, name='Y')

In [11]:
dropout_rate = tf.placeholder(shape=None, dtype=tf.float32)

In [12]:
with tf.variable_scope('Conv1') as scope:
    W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01), dtype=tf.float32, name='weight1')
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L1 = tf.nn.dropout(L1, rate=dropout_rate)
    print(L1)

Tensor("Conv1/dropout/mul_1:0", shape=(?, 14, 14, 32), dtype=float32)


In [13]:
with tf.variable_scope('Conv2') as scope:
    W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01), dtype=tf.float32, name='weight2')
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.nn.dropout(L2, rate=dropout_rate)
    print(L2)

Tensor("Conv2/dropout/mul_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [14]:
with tf.variable_scope('Conv3') as scope:
    W3 = tf.Variable(tf.random_normal(shape=[3, 3, 64, 128], stddev=0.01), dtype=tf.float32, name='weight3')
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L3 = tf.nn.dropout(L3, rate=dropout_rate)
    print(L3)

Tensor("Conv3/dropout/mul_1:0", shape=(?, 4, 4, 128), dtype=float32)


In [15]:
with tf.variable_scope('Fuuly-connected-1') as scope:
    L3 = tf.reshape(L3, shape=[-1, 4 * 4 * 128])
    W4 = tf.get_variable('weight4', shape=[4 * 4 * 128, 625], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal(shape=[625]), dtype=tf.float32, name='bias4')
    L4 = tf.matmul(L3, W4) + b4
    L4 = tf.nn.relu(L4)
    L4 = tf.nn.dropout(L4, rate=dropout_rate)
    print(L4)

Tensor("Fuuly-connected-1/dropout/mul_1:0", shape=(?, 625), dtype=float32)


In [16]:
with tf.variable_scope('Fuuly-connected-2') as scope:
    W5 = tf.get_variable('weight5', shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal(shape=[10]), dtype=tf.float32, name='bias5')
    hypothesis = tf.matmul(L4, W5) + b5
    print(hypothesis)

Tensor("Fuuly-connected-2/add:0", shape=(?, 10), dtype=float32)


In [17]:
with tf.variable_scope('cost'):
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y)
    cost = tf.reduce_mean(cost_i)

In [18]:
with tf.variable_scope('Optimizer'):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [19]:
with tf.variable_scope('accuracy') as scope:
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Instructions for updating:
Use `tf.math.argmax` instead


In [20]:
with tf.variable_scope('tensorboard') as scope:
    overall_cost     = tf.placeholder(shape=None, dtype=tf.float32, name='overall_cost')
    overall_accuracy = tf.placeholder(shape=None, dtype=tf.float32, name='overall_accuracy')
    tf.summary.scalar('cost', overall_cost)
    tf.summary.scalar('accuracy', overall_accuracy)

In [21]:
with tf.Session(config=config) as sess:
    # tensorboard --logdir=./logs/Deep_NN
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        _overall_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost, train], feed_dict={X: batch_xs, Y: batch_ys, dropout_rate: 0.3})
            _overall_cost += cost_val / total_batch
        
        _overall_accuracy = 0
        total_batch = int(mnist.test.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.test.next_batch(batch_size)
            accuracy_val = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys, dropout_rate: 0.0})
            _overall_accuracy += accuracy_val / total_batch
            
        summary = sess.run(merged_summary, feed_dict={overall_cost: _overall_cost, overall_accuracy: _overall_accuracy})
        writer.add_summary(summary, global_step=(epoch))
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(_overall_cost), 'accuracy =', '{:.9f}'.format(_overall_accuracy))

Epoch: 0001 cost = 0.378605048 accuracy = 0.979900009
Epoch: 0002 cost = 0.096698895 accuracy = 0.984100010
Epoch: 0003 cost = 0.075157092 accuracy = 0.989000010
Epoch: 0004 cost = 0.062308502 accuracy = 0.988900009
Epoch: 0005 cost = 0.051795872 accuracy = 0.990100009
Epoch: 0006 cost = 0.047791577 accuracy = 0.990600008
Epoch: 0007 cost = 0.043464535 accuracy = 0.992800006
Epoch: 0008 cost = 0.040883715 accuracy = 0.992300007
Epoch: 0009 cost = 0.036452873 accuracy = 0.992600006
Epoch: 0010 cost = 0.033624585 accuracy = 0.993400006
Epoch: 0011 cost = 0.033449110 accuracy = 0.991700007
Epoch: 0012 cost = 0.031015347 accuracy = 0.993000007
Epoch: 0013 cost = 0.030349024 accuracy = 0.994600005
Epoch: 0014 cost = 0.028060245 accuracy = 0.994100005
Epoch: 0015 cost = 0.025168537 accuracy = 0.992600007
Epoch: 0016 cost = 0.024807756 accuracy = 0.992100007
Epoch: 0017 cost = 0.024380552 accuracy = 0.993600006
Epoch: 0018 cost = 0.024561212 accuracy = 0.992900006
Epoch: 0019 cost = 0.0231886